Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
`https://github.com/squillero/computational-intelligence`  
Free for personal or classroom use; see 'LICENCE.md' for details.

In [1]:
from itertools import product
from collections import deque
import networkx as nx
import numpy as np

from tqdm.notebook import tqdm

In [2]:
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any((any(_contains_duplicates(sol[r,:]) for r in range(9)), 
                any(_contains_duplicates(sol[:,r]) for r in range(9)), 
                any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))))

def valid_solution(sol):
    return not contains_duplicates(sol) and np.sum(sol) == (1+2+3+4+5+6+7+8+9) * 9


In [3]:
SUDOKU = np.array([[6, 0, 4,    0, 7, 0,    0, 0, 1],
                   [0, 5, 0,    0, 0, 0,    0, 7, 0], 
                   [7, 0, 0,    5, 9, 6,    8, 3, 4], 

                   [0, 8, 0,    0, 0, 2,    4, 9, 0], 
                   [1, 0, 0,    0, 0, 0,    0, 0, 3], 
                   [0, 6, 9,    7, 0, 0,    0, 5, 0], 

                   [9, 1, 8,    3, 6, 7,    0, 0, 5], 
                   [0, 4, 0,    0, 0, 0,    0, 6, 0], 
                   [2, 0, 0,    0, 5, 0,    7, 0, 8]], dtype=np.byte)

In [4]:
frontier = deque([SUDOKU.copy()])

with tqdm(unit="n", total=None, unit_scale=True) as progress:
    tot_nodes = 0
    while frontier:
        tot_nodes += 1
        progress.update(1)

        node = frontier.popleft()

        if valid_solution(node):
            break

        stuck = True
        for i, j in zip(*np.where(node == 0)):
            for c in range(1, 10):
                node[i, j] = c
                if not contains_duplicates(node):
                    frontier.append(node.copy())
                    stuck = False

if valid_solution(node):
    print(f"Whoa! Solved expanding {tot_nodes:,} nodes")


0.00n [00:00, ?n/s]

Whoa! Solved expanding 1,526 nodes


In [5]:
node

array([[6, 9, 4, 8, 7, 3, 5, 2, 1],
       [8, 5, 3, 1, 2, 4, 9, 7, 6],
       [7, 2, 1, 5, 9, 6, 8, 3, 4],
       [3, 8, 5, 6, 1, 2, 4, 9, 7],
       [1, 7, 2, 9, 4, 5, 6, 8, 3],
       [4, 6, 9, 7, 3, 8, 1, 5, 2],
       [9, 1, 8, 3, 6, 7, 2, 4, 5],
       [5, 4, 7, 2, 8, 1, 3, 6, 9],
       [2, 3, 6, 4, 5, 9, 7, 1, 8]], dtype=int8)

In [6]:
sudoku = SUDOKU.copy()

tokens = list(range(1, 10)) * 9
for n in (_ for _ in sudoku.ravel() if _ != 0):
    tokens.remove(n)

n = 0
max_depth = 0
def backtrack(i):
    global n
    global max_depth
    n += 1

    if i > max_depth:
        max_depth = i
        print(f"Depth: {max_depth}")

    if contains_duplicates(sudoku):
        return False
    elif valid_solution(sudoku):
        print(f"Whoa! After {n} steps")
        return True
    for p in list(zip(*np.where(sudoku==0))):
        sudoku[p] = tokens[i]
        if backtrack(i+1):
            return True
        sudoku[p] = 0
    return False

found = backtrack(0)
if found:
    print(sudoku)

Depth: 1
Depth: 2
Depth: 3
Depth: 4
Depth: 5
Depth: 6
Depth: 7
Depth: 8
Depth: 9
Depth: 10
Depth: 11
Depth: 12
Depth: 13
Depth: 14
Depth: 15
Depth: 16
Depth: 17
Depth: 18
Depth: 19
Depth: 20
Depth: 21
Depth: 22
Depth: 23
Depth: 24
Depth: 25
Depth: 26
Depth: 27
Depth: 28
Depth: 29
Depth: 30
Depth: 31
Depth: 32
Depth: 33
Depth: 34
